### Youtube Sentence Adjust Word List Ratio

In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import glob

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
folder_name = "Turkish"
word_num = 28

#### Calculate Sentence Ratio By Word

In [4]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,ben,4908913
7,de,4880315
8,çok,4852169
9,ama,4661966


In [5]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [6]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [7]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"bunlar, büyük, normalde, kullanılıyor, özgür, ...","daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"tarafı, burcu, kolay, arka, bağlamak, olmayacak",
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"özgür, attıracağım, arabaya, yarım, tur",
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"biraz, burcu, tamam, devam, et, oldu",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"şimdilik, takacağız, artarsa, burcu, kalınlığı...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"ki, umarız, büyük, herkes, yanlış, ellerde, gü...","ve, için, bu"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"herkesin, özgün, gelişir, kurumlarda, eşit, ya...","bir, ve, daha"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"teşekkürler, zlediğiniz, i",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"açıklamada, link, olarak, kaynakları, belirtti...",


In [8]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"bunlar, büyük, normalde, kullanılıyor, özgür, ...","daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"tarafı, burcu, kolay, arka, bağlamak, olmayacak",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"özgür, attıracağım, arabaya, yarım, tur",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"biraz, burcu, tamam, devam, et, oldu",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"şimdilik, takacağız, artarsa, burcu, kalınlığı...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"ki, umarız, büyük, herkes, yanlış, ellerde, gü...","ve, için, bu"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"herkesin, özgün, gelişir, kurumlarda, eşit, ya...","bir, ve, daha"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"teşekkürler, zlediğiniz, i",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"açıklamada, link, olarak, kaynakları, belirtti...",NaN


In [9]:
df_sentence_ratio_result.to_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv", index=False)

#### Select Adjust Sentence Ratio Result

In [ ]:
df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
#df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
df_sentence_ratio_result

In [ ]:
adjust_sent_word_ratio = 100
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{word_num}_word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

In [ ]:
df_adjust_sentence_ratio.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv", index=False)

In [ ]:
df_adjust_sentence_ratio.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.xlsx", index=False)

#### Calculate Videoid Text Ratio By Word

In [10]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,ben,4908913
7,de,4880315
8,çok,4852169
9,ama,4661966


In [11]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [12]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence

,video_id,sentence
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [13]:
def videoid_word_ratio(df_word, df_videoid_sentence, word_column_name, videoid_sentence_column_name, word_num):
    word_set = set(df_word[f"{word_column_name}"].to_list())
    for i in range(len(df_videoid_sentence)):
        text = df_videoid_sentence.loc[i,f"{videoid_sentence_column_name}"]
        text_word = re.findall(r"\w+", text, re.UNICODE)
        text_word_set = set(text_word)
        intersect_word = list(word_set.intersection(text_word_set))
        different_word = list(text_word_set.difference(word_set))
        df_videoid_sentence.loc[i,f"{word_num}_word_ratio"] = round(((len(intersect_word)/len(text_word)+0.001)*100),)
        df_videoid_sentence.loc[i,"different_word"] = ", ".join(different_word)
        df_videoid_sentence.loc[i,"intersect_word"] = ", ".join(intersect_word)
        
    return df_videoid_sentence

In [14]:
df_text_ratio_result = videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.0,"belirteyim, merhaba, kuymağa, koyuyorum, ateşi...","bunu, kadar, çok, da, gibi, de, var, ama, için..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,2.0,"başarız, orman, başlanırsa, partili, ha, tamam...","onu, evet, de, ben, var, ama, bir, daha, şey, ..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.0,"l, isensiz, bahsetmedin, iyoksa, şu, hem, yaşa...","sen, ben, mi, de, ne"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,14.0,"kalıyor, diyorsun, ediyor, kendinle, koyuyorsu...","sen, kadar, çok, değil, onu, da, mi, gibi, var..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.0,"declared, unbelievable, method, group, they, m...",
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,3.0,"sağlığına, geldin, portakal, gelirimbağışıklık...","çok, bana, da, ben, gibi, de, iyi, var, ama, i..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,15.0,"öldürürüm, bırak, hiçkimse, anlatmaya, anlaşal...","bana, da, ben, mi, gibi, de, seni, bu"
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,1.0,"düşünürsek, otomatik, yatak, banyosu, bahsedec...","bana, onu, evet, de, ben, var, ama, beni, bir,..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,14.0,"büyük, bakabileceğim, küçük, reklamlardan, mem...","kadar, çok, ben, ve, için, bir"


In [15]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.0,"belirteyim, merhaba, kuymağa, koyuyorum, ateşi...","bunu, kadar, çok, da, gibi, de, var, ama, için..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,2.0,"başarız, orman, başlanırsa, partili, ha, tamam...","onu, evet, de, ben, var, ama, bir, daha, şey, ..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.0,"l, isensiz, bahsetmedin, iyoksa, şu, hem, yaşa...","sen, ben, mi, de, ne"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,14.0,"kalıyor, diyorsun, ediyor, kendinle, koyuyorsu...","sen, kadar, çok, değil, onu, da, mi, gibi, var..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.0,"declared, unbelievable, method, group, they, m...",NaN
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,3.0,"sağlığına, geldin, portakal, gelirimbağışıklık...","çok, bana, da, ben, gibi, de, iyi, var, ama, i..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,15.0,"öldürürüm, bırak, hiçkimse, anlatmaya, anlaşal...","bana, da, ben, mi, gibi, de, seni, bu"
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,1.0,"düşünürsek, otomatik, yatak, banyosu, bahsedec...","bana, onu, evet, de, ben, var, ama, beni, bir,..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,14.0,"büyük, bakabileceğim, küçük, reklamlardan, mem...","kadar, çok, ben, ve, için, bir"


In [16]:
df_text_ratio_result.to_csv(f"Youtube_Videoid_Text_{word_num}_Word_Ratio_Result.csv", index=False)

#### Select Adjust Videoid Text Ratio Result

In [20]:
df_text_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Videoid_Text_{word_num}_Word_Ratio_Result.csv")
#df_text_ratio_result = pd.read_csv(f"Youtube_Videoid_Text_{word_num}_Word_Ratio_Result.csv")
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.0,"şöyle, karadeniz, eridikten, kıvamında, ayarı,...","da, ama, var, ve, için, kadar, çok, bu, de, gi..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,2.0,"memnunlar, demokrasiye, beri, basın, batırdık,...","yok, ama, mı, çok, hayır, daha, bir, da, evet,..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.0,"isen, ölürümi, dur, bui, geçirsini, ievlat, io...","sen, ben, ne, de, mi"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,14.0,"durumunuzda, gece, nanın, abla, yabancı, bilmi...","beni, sen, da, ama, var, onu, ne, kadar, çok, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.0,"twilight, something, ground, today, risk, acce...",NaN
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,3.0,"pazardan, karadeniz, ederimah, geldin, yoruldu...","beni, iyi, gibi, da, ama, ben, var, ne, ve, iç..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,15.0,"yaklaşırsın, elimden, uzaklaş, dışında, kızını...","da, ben, bu, de, gibi, seni, mi, bana"
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,1.0,"şöyle, batımını, satacağız, kumsal, odasından,...","yok, ama, mı, çok, daha, bir, beni, da, evet, ..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,14.0,"videolara, büyük, reklamlardan, işler, vardı, ...","ben, ve, için, kadar, çok, bir"


In [21]:
adjust_videoid_text_word_ratio = 50
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result[f"{word_num}_word_ratio"] >= adjust_videoid_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,2LqpcYH8BVk,bu,100.0,NaN,bu
1,3ceSnmV4vPc,ben burada,50.0,burada,ben
2,CqQ7n02vocc,devamı var,50.0,devamı,var
3,GiJuh_GPHl0,bu köyün,50.0,köyün,bu
4,Jr3tr19glro,iyi seyirler,50.0,seyirler,iyi
5,Jv5n18MD32M,çok basit,50.0,basit,çok
6,S4F9FP1e2ps,ama sen çok pahalı bir kedi fiyatı,57.0,"pahalı, fiyatı, kedi","çok, sen, bir, ama"
7,WJY9mOwEFSE,karşınızda adam mı var hemen uygun bir pozisyo...,50.0,"karşınızda, uygun, pozisyonda, hemen, adam, vurun","mı, var, onu, kadar, bu, bir"
8,bw0DBP0oMkA,reklamlarınız için,50.0,reklamlarınız,için
9,pqKKGDNxyQ4,iyi seyirler,50.0,seyirler,iyi


In [ ]:
df_adjust_text_ratio.to_csv(f"Youtube_Videoid_{word_num}_Word_{adjust_videoid_text_word_ratio}_Ratio_Text.csv", index=False)

In [ ]:
df_adjust_text_ratio.to_excel(f"Youtube_Videoid_{word_num}_Word_{adjust_videoid_text_word_ratio}_Ratio_Text.xlsx", index=False)

#### Calculate Sequential Sentence Length Of Adjust Ratio Result

In [ ]:
df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
#df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].astype("object")
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].astype("object")

In [ ]:
df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
#df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

In [ ]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [ ]:
df_adjust_ratio["with_word"] = 1
df_adjust_ratio

In [ ]:
df_adjust_ratio_flag = pd.merge(df_sentence_ratio_result , df_adjust_ratio, how="left", on=["index","sentence","start_time","end_time","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_adjust_ratio_flag["with_word"].fillna(0, inplace=True)
df_adjust_ratio_flag

In [ ]:
df_adjust_ratio_flag.with_word.value_counts()

In [ ]:
# Code Running For Video_id Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_adjust_ratio_flag)):
    videoid = df_adjust_ratio_flag.loc[i,"video_id"]    
    if df_adjust_ratio_flag.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_adjust_ratio_flag.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [ ]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one.sort_values(by="length", ascending=False, inplace=True)
df_one.reset_index(drop=True, inplace=True)
df_one

In [ ]:
df_one_all = pd.DataFrame()
for i in range(len(df_one)):
    index = df_one.loc[i,"index"]
    length = df_one.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_all = pd.concat([df_one_all, df_video],axis=0)
df_one_all.reset_index(inplace=True, drop=True)
df_one_all

In [ ]:
#df_one_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentences_And_Others.xlsx", index=False)

In [ ]:
# Only Sequential Sentences
df_one_sequential = df_one[df_one.length > 1]
df_one_sequential

In [ ]:
df_one_sequential_all = pd.DataFrame()
for i in range(len(df_one_sequential)):
    index = df_one_sequential.loc[i,"index"]
    length = df_one_sequential.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_sequential_all = pd.concat([df_one_sequential_all, df_video],axis=0)
df_one_sequential_all.reset_index(inplace=True, drop=True)
df_one_sequential_all

In [ ]:
df_one_sequential_all.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.csv", index=False)

In [ ]:
df_one_sequential_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.xlsx", index=False)

#### Count Sentence Words By Adjust Column

In [ ]:
df_one_sequential_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
#df_one_sequential_all = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
df_one_sequential_all

In [ ]:
def text_to_list_df(df_target, target_column): # df_target include dict value columns, target_column occurs dict
    text_value_list = []
    for i in df_target[f"{target_column}"]:
        try:
            var = re.findall(r"\w+", i)
            for j in var:
                text_value_list.append(j)
        except:
            pass
    df_alone_value = pd.DataFrame(text_value_list)
    df_alone_value.rename(columns={0:f"{target_column}_alone"}, inplace=True)
    df_unique_count = pd.DataFrame(df_alone_value.value_counts(ascending=False))
    df_unique_count.reset_index(inplace=True)
    df_unique_count.rename(columns={0:"count"}, inplace=True)
            
    return df_unique_count

In [ ]:
df_unique_word_count = text_to_list_df(df_one_sequential_all, "intersect_word")
df_unique_word_count

In [ ]:
#df_unique_word_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentence_Words_Count.xlsx", index=False)

#### Count Adjust Sentence Ratio Result By Videoid

In [ ]:
df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
#df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

In [ ]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [ ]:
df_ratio_count = pd.DataFrame(df_adjust_ratio.groupby("video_id")[f"{word_num}_word_ratio"].count())
df_ratio_count.reset_index(inplace=True)
df_ratio_count.rename(columns={f"{word_num}_word_ratio":f"{word_num}_word_ratio_count"}, inplace=True)
df_ratio_count.sort_values(by=f"{word_num}_word_ratio_count", ascending=False, inplace=True)
df_ratio_count.reset_index(drop=True,inplace=True)
df_ratio_count

In [ ]:
#df_ratio_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Videoid_Count.xlsx", index=False)

#### Find Word In Sentence Time Interval

In [ ]:
df_word_count_low = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/{level.lower().capitalize()}/3-Lesson Level Result/Lesson_{level.lower().capitalize()}_Low_Count_Words.xlsx")
df_word_count_low

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
word_list = df_word_count_low.iloc[:,0].values.tolist()

In [ ]:
df_word_result = pd.DataFrame()
for i in word_list:
    try:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(1)
    except:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(1) 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

In [ ]:
word_time_loc_list = []
def word_time_loc(df):
    for i in range(len(df)):
        word = df.loc[i,"word"]
        start_time = df.loc[i,"start_time"]
        end_time = df.loc[i,"end_time"]
        sentence = df.loc[i,"sentence"]
        video_id = df.loc[i,"video_id"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=["word","start_time","end_time","sentence","video_id"])

    return df_word_time_loc

In [ ]:
df_word_time_loc_result = word_time_loc(df_word_result)
df_word_time_loc_result

In [ ]:
time_shift = 0.3
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_time_loc_result

In [ ]:
#df_word_time_loc_result.to_excel(f"Youtube_Sentence_{word_num}_Low_Word_Time_Interval.xlsx", index=False)